## Analysing output of PrEWRunRK fits

Using output of test toy fits from PrEW, import it into the PrOut python classes and create some histograms with it.

### Import PrEW output module

In [1]:
import sys
sys.path.append("../../PrEW/source/prout")
import PrOut

### Import other modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt

### Read output file

In [3]:
reader = PrOut.Reader("../output/fit_results.out")
reader.read()
print("Found " + str(len(reader.run_results)) + " run(s).")

Found 1 run(s).


### Collect parameters

In [4]:
# Assuming only one toy run (with many toys) was performed, get number of parameters and performed toy fits
result = reader.run_results[0]
n_pars = len(result.par_names)
n_fits = len(result.fit_results)

# Get the chi-squared values, parameters, uncertainties and correlation matrices from each fit
chi_sqs = np.empty(n_fits)
par_results = np.empty((n_pars,n_fits))
unc_results = np.empty((n_pars,n_fits))
cor_matrix_results = np.empty((n_pars,n_pars,n_fits))
for f in range(n_fits):
    chi_sqs[f] = result.fit_results[f].chisq_fin
    for p in range(n_pars):
        par_results[p][f] = result.fit_results[f].pars_fin[p]
        unc_results[p][f] = result.fit_results[f].uncs_fin[p]
        for p2 in range(n_pars):
            cor_matrix_results[p][p2][f] = result.fit_results[f].cor_matrix[p][p2]

### Calculate result averages

In [5]:
chi_sq_avg = np.average(chi_sqs)
par_avg = [np.average(p_vals) for p_vals in par_results]
unc_avg = [np.average(p_uncs) for p_uncs in unc_results]
cor_matrix_avg = np.array([[np.average(p1p2_element) for p1p2_element in p1_column] for p1_column in cor_matrix_results] )

### Create histograms for each parameter using boost-histogram

In [6]:
for p in range(n_pars):
    p_avg = par_avg[p] # Average result of current parameter
    p_min = np.amin(par_results[p]) # Smallest result for current parameter
    p_max = np.amax(par_results[p]) # Largest result for current parameter
    p_std = np.std(par_results[p],ddof=1) # Standard deviation of results for current parameter over all fits
    
    # x-axis minimum and maximum
    x_min = p_min - 0.1*(p_avg-p_min)
    x_max = p_max + 0.1*(p_max-p_avg)

    # Create a histogram for this parameter
    fig, ax = plt.subplots(tight_layout=True)
    hist_entries, hist_edges, _ = ax.hist(par_results[p], range=(x_min,x_max), bins=10, histtype='step', color='black', label="Fit results")
    ax.set_xlabel("Fit result")
    ax.set_ylabel("#Toys")
    
    # Calculate bin centers
    bin_centers = 0.5 * (hist_edges[:-1] + hist_edges[1:])
    
    # Plot poissonian errorbars
    ax.errorbar(bin_centers, hist_entries, yerr=np.sqrt(hist_entries), color='black', fmt='none')
    
    # Plot mean, standard deviation and average uncertainty
    y_lims = ax.get_ylim() # Keep y-axis limits constant
    l_mean = ax.plot([p_avg,p_avg], [0,y_lims[1]],color='red', label="Mean of toys")
    l_std_min = ax.plot([p_avg-p_std,p_avg-p_std], [0,y_lims[1]],color='green', label="Std. of toys")
    l_std_max = ax.plot([p_avg+p_std,p_avg+p_std], [0,y_lims[1]],color='green')
    l_unc_min = ax.plot([p_avg-unc_avg[p],p_avg-unc_avg[p]], [0,y_lims[1]],color='magenta', label="Avg. fit unc.")
    l_unc_max = ax.plot([p_avg+unc_avg[p],p_avg+unc_avg[p]], [0,y_lims[1]],color='magenta')
    ax.set_ylim(y_lims) # Keep y-axis limits constant
    
    # Add legend
    ax.legend(title=result.par_names[p])
    
    fig.savefig("../output/hist_" + result.par_names[p] + ".pdf")
    print(result.par_names[p] + " avg: " + str(p_avg) + " std: " + str(p_std) + " unc: " + str(unc_avg[p]))

Delta-g1Z avg: 0.0005822506 std: 0.00281941929146 unc: 0.00170793085
Delta-kappa_gamma avg: 0.00027854785 std: 0.000634932618305 unc: 0.00347488975
Delta-lambda_gamma avg: 0.00055036854 std: 0.00198505402203 unc: 0.0013279658
ChiXS_singleWplussemileptonic_LR avg: 1.00021523 std: 0.000683022724219 unc: 0.0031645749
ChiXS_singleWplussemileptonic_RL avg: 0.98777635 std: 0.0586875293861 unc: 0.0583360465
ChiXS_singleWplussemileptonic_RR avg: 1.000142805 std: 0.00114932429107 unc: 0.0081499086
ChiXS_singleWminussemileptonic_LR avg: 0.99982989 std: 0.000743890475944 unc: 0.00316708615
ChiXS_singleWminussemileptonic_RL avg: 1.0058629 std: 0.0488368955281 unc: 0.058753859
ChiXS_singleWminussemileptonic_LL avg: 0.99434431 std: 0.00799849492393 unc: 0.0106115465
ChiXS_WW_semilep_MuAntiNu_LR avg: 0.985770995 std: 0.0414008262668 unc: 0.026346183
ChiXS_WW_semilep_MuAntiNu_RL avg: 1.31339945 std: 0.281789618345 unc: 0.47806343
ChiXS_WW_semilep_AntiMuNu_LR avg: 0.962400385 std: 0.0464176923108 unc: 

/home/jakob/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


ePol- avg: 0.799835755 std: 0.000100204101962 unc: 0.000493344625
ePol+ avg: 0.800164375 std: 0.000189200561442 unc: 0.000239678145
pPol- avg: 0.299941835 std: 8.70660579675e-05 unc: 0.000397499765
pPol+ avg: 0.30014189 std: 0.000474892914245 unc: 0.000486673325


### Draw average correlation matrix

In [7]:
# Create figure for plots and adjust height and width
fig_cor, ax_cor = plt.subplots()
fig_cor.set_figheight(8.5)
fig_cor.set_figwidth(10)

# Plot a 2D color plot (sharp, no interpolations, using PRGn color map)
im_cor = ax_cor.imshow(cor_matrix_avg,interpolation='none',cmap='PRGn')
cb_cor = fig_cor.colorbar(im_cor, ax=ax_cor) # Show what the colors mean
im_cor.set_clim(-1, 1); # Correlations are between -1 and +1

# We want to show all ticks...
ax_cor.set_xticks(np.arange(n_pars))
ax_cor.set_yticks(np.arange(n_pars))
# ... and label them with the respective list entries
ax_cor.set_xticklabels(result.par_names)
ax_cor.set_yticklabels(result.par_names)

# Rotate the tick labels and set their alignment.
plt.setp(ax_cor.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

ax_cor.set_title("Average correlation matrix")
fig_cor.tight_layout()
fig_cor.savefig("../output/hist_cor.pdf")

/home/jakob/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


### Plot the chi-sq distribution

In [8]:
chi_sq_min = np.amin(chi_sqs) # Smallest chi^2 found
chi_sq_max = np.amax(chi_sqs) # Largest chi^2 found

# x-axis minimum and maximum
x_min_chi_sq = chi_sq_min - 0.1*(chi_sq_avg-chi_sq_min) 
x_max_chi_sq = chi_sq_max + 0.1*(chi_sq_max-chi_sq_avg)

# Plot histogram of all found chi^2 values
fig_chi_sq, ax_chi_sq = plt.subplots(tight_layout=True)
hist_chi_sq = ax_chi_sq.hist(chi_sqs, range=(x_min_chi_sq,x_max_chi_sq), bins=15, histtype='step', color='black')
ax_chi_sq.set_xlabel(r"$\chi^2$")
ax_chi_sq.set_ylabel("#Toys")

# Plot mean chi^2 of all toy fits
y_lims_chi_sq = ax_chi_sq.get_ylim()
l_mean_chi_sq = ax_chi_sq.plot([chi_sq_avg,chi_sq_avg], [0,y_lims_chi_sq[1]],color='red')
ax_chi_sq.set_ylim(y_lims_chi_sq)

fig_chi_sq.savefig("../output/hist_chisq.pdf")
print(r"Average $\chi^2$: " + str(chi_sq_avg))

/home/jakob/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Average $\chi^2$: 58255.488
